In [1]:
# This file costructs surrogate models for the input datasets
import numpy as np   
import pandas as pd
import os
import shutil
import json
import math
import time
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from joblib import Parallel, delayed, dump

# Torch specific module imports
import torch
import gpytorch 

# botorch specific modules
from botorch.fit import fit_gpytorch_model
from botorch.models.gpytorch import GPyTorchModel
from botorch.optim import optimize_acqf, optimize_acqf_discrete
from botorch import fit_gpytorch_mll
from botorch.acquisition.monte_carlo import (
    qExpectedImprovement,
    qNoisyExpectedImprovement,
)
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.acquisition import UpperConfidenceBound, ExpectedImprovement

# Plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Tick parameters
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1

plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['legend.fontsize'] = 15

# User defined python classes and files
import input_class 
import code_inputs as model_input
import utils_dataset as utilsd
import surrogate_models
import kmeans as km

# Set the random seeds
np.random.seed(0)
torch.manual_seed(0)

/Users/maitreyeesharma/opt/anaconda3/envs/torch/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/maitreyeesharma/opt/anaconda3/envs/torch/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: /Users/maitreyeesharma/opt/anaconda3/envs/torch/lib/python3.11/site-packages/torchvision/image.so
  Expected in: /Users/maitreyeesharma/opt/anaconda3/envs/torch/lib/python3.11/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Using cpu device


#### K means clustering

In [2]:
Input = input_class.inputs(input_path='../')
XX_prop, YY, descriptors = Input.read_inputs()
descriptors

['dimensions',
 ' bond type',
 ' void fraction [widom]',
 ' supercell volume [A^3]',
 ' density [kg/m^3]',
 ' heat desorption high P [kJ/mol]',
 ' absolute methane uptake high P [molec/unit cell]',
 ' absolute methane uptake high P [mol/kg]',
 ' excess methane uptake high P [molec/unit cell]',
 ' excess methane uptake high P [mol/kg]',
 ' heat desorption low P [kJ/mol]',
 ' absolute methane uptake low P [molec/unit cell]',
 ' absolute methane uptake low P [mol/kg]',
 ' excess methane uptake low P [molec/unit cell]',
 ' excess methane uptake low P [mol/kg]',
 ' surface area [m^2/g]',
 ' linkerA',
 ' linkerB',
 ' net',
 ' cell_a [A]',
 ' cell_b [A]',
 ' cell_c [A]',
 ' alpha [deg]',
 ' beta [deg]',
 ' gamma [deg]',
 ' num carbon',
 ' num fluorine',
 ' num hydrogen',
 ' num nitrogen',
 ' num oxygen',
 ' num sulfur',
 ' num silicon',
 ' vertices',
 ' edges',
 ' genus',
 ' largest included sphere diameter [A]',
 ' largest free sphere diameter [A]',
 ' largest included sphere along free sphe

In [3]:
XX_comp_df, YY_df = Input.get_comp()
XX_comp_df

,num carbon,num fluorine,num hydrogen,num nitrogen,num oxygen,num sulfur,num silicon
0,360,0,216,144,72,0,0
1,360,0,216,144,144,0,0
2,432,0,360,144,72,0,0
3,360,0,144,216,216,0,0
4,360,0,144,216,216,0,0
...,...,...,...,...,...,...,...
69835,996,0,576,96,0,0,0
69836,1020,0,576,48,0,0,0
69837,1360,0,768,64,0,0,0
69838,1888,0,1152,128,128,0,0


In [4]:
num_cluster = 3
clustered_dfs = km.k_means(XX_comp_df, YY_df, num_cluster)
sample_dfs = km.draw_samples(clustered_dfs, sample_fraction = 0.01)
samples = km.concat(sample_dfs)
samples

/Users/maitreyeesharma/opt/anaconda3/envs/torch/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,num carbon,num fluorine,num hydrogen,num nitrogen,num oxygen,num sulfur,num silicon,deliverable capacity [v STP/v]
0,1152,0,1024,256,0,0,32,145.407912
1,1152,0,1216,256,0,0,0,95.429169
2,768,0,512,0,0,0,32,168.557294
3,1128,0,576,192,192,0,0,115.141985
4,756,0,486,108,18,0,0,116.641903
...,...,...,...,...,...,...,...,...
692,1792,0,768,0,0,0,0,193.030421
693,2304,0,1392,288,96,0,0,107.276586
694,1620,0,1080,360,0,0,0,176.029207
695,1664,0,1024,640,512,0,0,164.708736


#### Acquisition function 

In [5]:
## TODO: TO BE Check
bounds = torch.tensor([[-10.0], [12.0]])

batch_size = 1
num_restarts= 10 
raw_samples = 512

def optimize_acqf_and_get_observation(acq_func, X_test, Y_test):
    """Optimizes the acquisition function, and returns a new candidate"""
    # optimize
    candidates, _ = optimize_acqf_discrete(
        acq_function=acq_func,
        choices=X_test,
        q=batch_size,
        max_batch_size=2048,
        num_restarts=num_restarts,
        raw_samples=raw_samples,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        unique=True
    )
    
    print(candidates)
    # observe new values
    new_x = candidates.detach()
    b = [1 if torch.all(X_test[i].eq(new_x)) else 0 for i in range(0,X_test.shape[0]) ]
    b = torch.tensor(b).to(torch.int)
    index = b.nonzero()[0][0]
    new_y = torch.reshape(Y_test[0,index],(1,1))
    
    X_test_new = X_test[torch.arange(0, X_test.shape[0]) != index, ...]
    Y_test_new = Y_test[..., torch.arange(0, Y_test.shape[1]) != index]
    print(X_test_new)
    print(Y_test_new)
    
    return new_x, new_y, index, X_test_new, Y_test_new

#### GP Train Function

In [6]:
def create_train_test_data(cluster_dataXX, cluster_dataYY, random_seed):
    if model_input.standardize_data:
        cluster_dataXX, scalerX_transform = utilsd.standardize_data(cluster_dataXX)
        cluster_dataYY, scalerY_transform = utilsd.standardize_data(cluster_dataYY.reshape(-1,1))
    else:
        scalerX_transform = None
        scalerY_transform = None
    
    ## TODO : Incase for feature selection
        # ....
        # ....
        # ....

    # Create train and test sets
    X_train, X_test, Y_train, Y_test = train_test_split(cluster_dataXX, cluster_dataYY, test_size=model_input.test_size, random_state=random_seed)

    # Convert to tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    Y_train = np.transpose(Y_train) # IMP : Has to have only one row for GP training
    Y_train = torch.tensor(Y_train, dtype=torch.float32)
    Y_test = np.transpose(Y_test)
    Y_test = torch.tensor(Y_test, dtype=torch.float32)

    return X_train, X_test, Y_train, Y_test, scalerX_transform, scalerY_transform

def train_gp(cluster_idx):
    best_observed_all_ei0 = []
    # Average over multiple trials
    for trial in range(1, model_input.n_trials + 1):
        t0 = time.monotonic()
        if model_input.random_seed == 'time':
            random_seed = int(t0)
        elif model_input.random_seed == 'iteration':
            random_seed = trial
            
        print(f"\n -------------------- Trial {trial:>2} of {model_input.n_trials} --------------------\n", end="")
        best_observed0 = []

        XX_desc = list(sample_dfs[cluster_idx].columns[:-1])
        YY_desc = sample_dfs[cluster_idx].columns[-1]
        (
            X_train,
            X_test,
            Y_train,
            Y_test,
            scalerX, 
            scalerY
        ) = create_train_test_data(sample_dfs[cluster_idx][XX_desc].to_numpy(), sample_dfs[cluster_idx][YY_desc].to_numpy(), random_seed)
        if trial == 1:
            # Dump the scalers to model output folder
            dump(scalerX, os.path.join(model_input.output_folder, f'scalerX_{cluster_idx}.joblib'))
            dump(scalerY, os.path.join(model_input.output_folder, f'scalerY_{cluster_idx}.joblib'))
            
        # Finding best value in initial data
        if model_input.maximization:
            best_observed_value = Y_train.max()
            optimal_solution = torch.cat([Y_train[0],Y_test[0]]).max()
        else:
            best_observed_value = Y_train.min()
            optimal_solution = torch.cat([Y_train[0],Y_test[0]]).min()
        
        # If optimal value is present in the initial dataset sample remove it  
        if (best_observed_value.eq(optimal_solution)) and model_input.maximization:
            print('Max in training set, removing it before training models.')
            optimal_position = torch.argmax(Y_train)
            
            # Add max value to test/exploration set
            X_add_toTest = torch.reshape(X_train[optimal_position,:],(1,X_train.shape[1]))
            X_test = torch.cat([X_test,X_add_toTest])
            Y_add_toTest = torch.reshape(optimal_solution,(1,1))      
            Y_test = torch.cat((Y_test,Y_add_toTest),1)
            
            # Remove max value from training set
            X_train = X_train[torch.arange(0, X_train.shape[0]) != optimal_position, ...]
            Y_train = Y_train[..., torch.arange(0, Y_train.shape[1]) != optimal_position]
            
            # Update best observed value
            best_observed_value = Y_train.max()
            
        elif (best_observed_value.eq(optimal_solution)) and not model_input.maximization:
            print('Min in training set, removing it before training models.')
            optimal_position = torch.argmin(Y_train)
            
            # Add min value to test/exploration set
            X_add_toTest = torch.reshape(X_train[optimal_position,:],(1,X_train.shape[1]))
            X_test = torch.cat([X_test,X_add_toTest])
            Y_add_toTest = torch.reshape(optimal_solution,(1,1))      
            Y_test = torch.cat((Y_test,Y_add_toTest),1)
            
            # Remove min value from training set
            X_train = X_train[torch.arange(0, X_train.shape[0]) != optimal_position, ...]
            Y_train = Y_train[..., torch.arange(0, Y_train.shape[1]) != optimal_position]
            
            # Update best observed value
            best_observed_value = Y_train.min()
        
        # Initialize data for training gp-0 and gp-l models
        X_train0, Y_train0, X_test0, Y_test0 = X_train, Y_train, X_test, Y_test
                
        n_batch = model_input.n_batch_perTrial
        
        # Initialize likelihood, GP model and acquisition function for the models
        #--------------------------- GP-0 ---------------------------#
        likelihood_gp0 = gpytorch.likelihoods.GaussianLikelihood()
        model_gp0 = surrogate_models.ExactGPModel(X_train0, Y_train0, likelihood_gp0)           
        AcqFunc_0 = ExpectedImprovement(model=model_gp0, best_f=best_observed_value, maximize=model_input.maximization)
        best_observed0.append(best_observed_value)  # Appending to best_observed list for the given trial
        
        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, n_batch + 1):

            if ((iteration-1)%model_input.n_update==0):
                # fit the models every 10 iterations
                model_gp0, likelihood_gp0 = surrogate_models.train_surrogate_gp0(X_train0, Y_train0)
        
            # optimize and get new observation using acquisition function
            new_x0, new_y0, index, X_test_new0, Y_test_new0 = optimize_acqf_and_get_observation(AcqFunc_0, X_test0, Y_test0)
            
            # Update remaining choices tensor
            X_test0 = X_test_new0
            Y_test0 = Y_test_new0

            # Update training points
            X_train0 = torch.cat([X_train0, new_x0])
            Y_train0 = torch.cat([Y_train0[0], new_y0[0]])
            Y_train0 = torch.reshape(Y_train0,(1,Y_train0.shape[0]))

            # update progress
            if model_input.maximization:
                best_value_ei0 = Y_train0.max()
            elif not model_input.maximization:
                best_value_ei0 = Y_train0.min()
            best_observed0.append(best_value_ei0)

            # AcqFunc_0 = UpperConfidenceBound(model_gp0, beta=0.1) 
            AcqFunc_0 = ExpectedImprovement(model=model_gp0, best_f=best_value_ei0, maximize=model_input.maximization)
        
            if model_input.verbose:
                print(
                    f"\nBatch {iteration:>2}: best_value (GP-0) = ",
                    f"({best_value_ei0:>4.2f}",
                    end="",)

        t1 = time.monotonic()
        
        print(f"time = {t1-t0:>4.2f}.")
        # Appending to common list of best observed values, with number of rows equal to number of trials
        best_observed_all_ei0.append(best_observed0) 
    return best_observed_all_ei0

#### Main Function

In [7]:
warnings.filterwarnings("ignore", category=BadInitialCandidatesWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Create a new directory if it does not exist
isExist = os.path.exists(model_input.output_folder)
if not isExist:
    os.makedirs(model_input.output_folder)
    print("The new directory is created!", model_input.output_folder)
    
# Commented out by NKT
# # Copy input parameters file to output folder
# shutil.copy2('surrogate_model_inputs.py',model_input.output_folder)
# Copy surrogate model file to output folder
shutil.copy2('surrogate_models.py',model_input.output_folder)

# Training a single GP for test
# a = train_gp(0)

# Train the cluster of GP models in a parallel for loop
best_observed_all_ei0 = Parallel(n_jobs=-1)(
    delayed(train_gp)(i) for i in range(num_cluster)
)

/Users/maitreyeesharma/opt/anaconda3/envs/torch/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(

KeyboardInterrupt



Error in callback <bound method AutoreloadMagics.post_execute_hook of <IPython.extensions.autoreload.AutoreloadMagics object at 0x168e0be90>> (for post_execute):



KeyboardInterrupt



In [ ]:
# Epsilon greedy method:

random_number = np.random.rand()
# Explore using the Epsilon Greedy Exploration Strategy
if random_number <= epsilon:
    # Explore
    best_observed = ## replace here with the randomly picking out one cluster's BO recommendation
else:
    # Exploit best known action
    best_observed = ## replace here with the best output from the BO recommedations of the local GPs
    